## Submit Content for a Twig Branch

This notebook creates a submit for an existing branch that accepts direct edits (a twig)

In [1]:
import os 
os.chdir('..')

In [6]:
import lakat.branch.functions as lakat_branch_functions
import lakat.branch.schema as lakat_branch_schema
import lakat.submit.functions as lakat_submit_functions
import lakat.submit.schema as lakat_submit_schema
import lakat.storage as lakat_storage_functions
from utils.format.schema import check_argument, convert_to_bytes_based_on_schema, convert_from_bytes_based_on_schema
from config.branch_cfg import PROPER_BRANCH_TYPE_ID, TWIG_BRANCH_TYPE_ID
from utils.encode.bytes import encode_bytes_to_base64_str, decode_base64_str_to_bytes
# from config.encode_cfg import ENCODING_FUNCTION
from config.bucket_cfg import DEFAULT_ATOMIC_BUCKET_SCHEMA, DEFAULT_MOLECULAR_BUCKET_SCHEMA, BUCKET_ID_TYPE_NO_REF
# from utils.encode.bytes import encode_bytes_to_base64_str, decode_base64_str_to_bytes

In [3]:
# some fake signature encoded in bytes64 
signature = encode_bytes_to_base64_str(bytes(0))
accept_conflicts = False
msg = 'Genesis Submit'
name = 'Genesis Branch'
kwargs = dict(branch_type=TWIG_BRANCH_TYPE_ID, name=name, signature=signature, accept_conflicts=accept_conflicts, msg=msg)
print('keyword arguments: ', kwargs)

keyword arguments:  {'branch_type': 1, 'name': 'Genesis Branch', 'signature': '', 'accept_conflicts': False, 'msg': 'Genesis Submit'}


### First create Genesis Branch (Twig)

In [4]:
check_argument(arg=kwargs, schema=lakat_branch_schema.create_genesis_branch_call)
converted_kwargs = convert_to_bytes_based_on_schema(schema=lakat_branch_schema.create_genesis_branch_call, data=kwargs)
response = lakat_branch_functions.create_genesis_branch(**converted_kwargs)
decoded_response = convert_from_bytes_based_on_schema(schema=lakat_branch_schema.create_genesis_branch_response, data=response)

### Prepare Submit Content

In [5]:
contents = [
        {
            "data": "Hello",  
            "schema": DEFAULT_ATOMIC_BUCKET_SCHEMA, 
            "parent_id": encode_bytes_to_base64_str(bytes(0)), 
            "signature": encode_bytes_to_base64_str(bytes(1)), 
            "refs": []
        },
        {
            "data": "World",  
            "schema": DEFAULT_ATOMIC_BUCKET_SCHEMA, 
            "parent_id": encode_bytes_to_base64_str(bytes(0)), 
            "signature": encode_bytes_to_base64_str(bytes(1)), 
            "refs": []
        },
        {
            "data": {
                "order": [
                    {"id": 0, "type": BUCKET_ID_TYPE_NO_REF},
                    {"id": 1, "type": BUCKET_ID_TYPE_NO_REF}], 
                "name": "Dummy Article Name"},
            "schema": DEFAULT_MOLECULAR_BUCKET_SCHEMA,
            "parent_id": encode_bytes_to_base64_str(bytes(0)), 
            "signature": encode_bytes_to_base64_str(bytes(1)), 
            "refs": []
        }]

In [9]:
public_key = encode_bytes_to_base64_str(bytes(1))
proof = encode_bytes_to_base64_str(bytes(1))
submit_msg = "Initial Submit"
submit_kwargs = dict(branch_id=decoded_response, contents=contents, public_key=public_key, proof=proof, msg=submit_msg)

### Submit Content for a Twig without RPC server call

In [10]:
check_argument(arg=submit_kwargs, schema=lakat_submit_schema.submit_content_for_twig_call)
converted_submit_kwargs = convert_to_bytes_based_on_schema(schema=lakat_submit_schema.submit_content_for_twig_call, data=submit_kwargs)
submit_response = lakat_submit_functions.submit_content_for_twig(**converted_submit_kwargs)
decoded_submit_response = convert_from_bytes_based_on_schema(schema=lakat_submit_schema.submit_content_for_twig_response, data=submit_response)

In [11]:
print('original submit response: ', submit_response ,'\ndecoded submit response: ', decoded_submit_response)

original submit response:  b'\x01Q\x12\x02\xb1z\n\x04\x8c-\x00\x00' 
decoded submit response:  AVESArF6CgSMLQAA


### Submit Content for a Twig via RPC server call

First start the server via `python main.py` (first need to install the dependencies from the *requirements.txt*) or start the docker container exposed on port `3355`.

In [12]:
import requests
import json
from config.rpc_cfg import RPC_PORT

In [13]:
def json_rpc_call(method, params=None):
    url = f"http://localhost:{RPC_PORT}/"
    headers = {'content-type': 'application/json'}
    payload = {
        "method": method,
        "params": params,
        "jsonrpc": "2.0",
        "id": 0,
    }
    encoded_payload = json.dumps(payload).encode('utf-8')
    response = requests.post(url, data=encoded_payload, headers=headers)
    # print(response)#
    return response.json()

In [14]:
submit_kwargs_rpc = list(submit_kwargs.values())
response = json_rpc_call(method="submit_content_to_twig", params=submit_kwargs_rpc)
# response is inside the result key
branch_head_id = response['result']
print("Test 'submit_content_to_twig' passed with branch head ID:", branch_head_id)

Test 'submit_content_to_twig' passed with branch head ID: AVESAmKWCgSMLQAA
